In [1]:
import pandas as pd
import xml.etree.ElementTree as ET

In [2]:
import os, glob, shutil, xnat, pyxnat

In [3]:
tree = ET.parse('/home/hdary/PROTOCOLE_CERVEAU.xml')
root = tree.getroot()

In [4]:
dico_sequences = {}
for NormalStep_decision_branch in root.iter('NormalStep_decision_branch'):
    dico_sequences[NormalStep_decision_branch.attrib.get('Id')] = NormalStep_decision_branch.attrib.get('name')

In [5]:
dico_sequences

{'eda9e1cb-46de-4f8f-bdcd-162feaaebbbb': 'localizer',
 '720b5d00-767b-45d1-b05a-e77e7e960c4e': 'gre_b0map_sag_p2',
 '9c237fd3-34db-4be6-b991-b016b794cd3f': 'fid_B1Mapping',
 '22cbba65-2abe-4e59-a878-828e619bafec': 'DA_3D_RAD_8TE_bloc1_hamming',
 'eaaf56e2-bd37-4a27-b056-fba077e5bb68': 'DA_3D_RAD_8TE_bloc2_hamming',
 'f876d8a8-beea-4d38-b417-fadfc4c08ae3': 'DA_3D_RAD_8TE_bloc3_hamming',
 '750fddad-ff20-4046-8716-d24b9c6fb74b': 'AAHead_Scout_32ch-head-coil',
 '47ee06a8-ae59-4464-9eb2-dbd130d5ae17': 'tfl_b1map_sag_3mm',
 '948911c5-d2e6-4a8e-bb42-87db5c9f0523': 'gre_fieldmap_sag_3mm',
 '378e0b41-f9f4-4650-9ee0-e691c2fb9025': 't1_mp2rage_sag_0p6',
 '54f2f934-02fe-4efc-aff5-363ab572db68': 't2_space_flair_sag_0p8',
 'ab4c0ae7-d13a-4863-ac82-dc541672cd34': 't2_tse_tra_CACP',
 'dd63aa5d-2c90-4a43-8e1c-c5b29ed3cea2': 'mbep2d_bold_tra_LR_HCP_1p6',
 'b9628d91-9a2a-473e-ba1a-98531e7b9132': 'mbep2d_bold_tra_RL_HCP_1p6',
 'd0639cab-3dd8-42e9-a6b5-d713e4cbb507': 'mbep2d_DTI_tra_AP_80dir_1p13',
 '6c287

In [6]:
dico_sequences_attributes = {}

for child in root[1:]:
    for child2 in child:
        sequence_id = child2.attrib.get('Id')
        print(dico_sequences[sequence_id])
        dico_sequences_attributes[dico_sequences[sequence_id]] = {}
        for child3 in child2:
            for child4 in child3[1:]:
                dico_sequences_attributes[dico_sequences[sequence_id]][child4.attrib.get('name')] = {}
                for child5 in child4:
                    # for child6 in child5:
                    #     print(child6.attrib, child6.tag)
                    valueAndUnit_list = []
                    label_list = []
                    for valueAndUnit in child5.iter('ValueAndUnit'):
                        valueAndUnit_list.append(valueAndUnit.text)

                    for label in child5.iter('Label'):
                        label_list.append(label.text) 
                    
                    for i,j in enumerate(label_list):
                        dico_sequences_attributes[dico_sequences[sequence_id]][child4.attrib.get('name')][j] = valueAndUnit_list[i]

        # for neighbor in root.iter('ProtParameter'):
        #     print(neighbor.attrib)
        print(dico_sequences_attributes[dico_sequences[sequence_id]])
        print('-----------------------------------')


localizer
{'Properties': {'Prio recon': 'On ', 'Load images to viewer': 'On ', 'Inline movie': 'Off ', 'Auto store images': 'On ', 'Load images to stamp segments': 'On ', 'Load images to graphic segments': 'On ', 'Auto open inline display': 'Off ', 'Auto close inline display': 'Off ', 'Start measurement without further preparation': 'On ', 'Wait for user to start': 'Off ', 'Start measurements': 'Single measurement '}, 'Routine': {'Slice group': '3', 'Slices': '1 ', 'Dist. factor': '20 %', 'Position': 'Isocenter ', 'Orientation': 'Coronal ', 'Phase enc. dir.': 'R >> L ', 'AutoAlign': '--- ', 'Phase oversampling': '0 %', 'FoV read': '250 mm', 'FoV phase': '100.0 %', 'Slice thickness': '5.0 mm', 'TR': '8.6 ms', 'TE': '3.69 ms', 'Averages': '2 ', 'Concatenations': '3 ', 'Filter': 'Elliptical filter ', 'Coil elements': '1H '}, 'Contrast - Common': {'TR': '8.6 ms', 'TE': '3.69 ms', 'TD': '0 ms', 'MTC': 'Off ', 'Magn. preparation': 'None ', 'Flip angle': '20 deg', 'Fat suppr.': 'None ', 'Wate

In [7]:
df = pd.DataFrame.from_dict(dico_sequences_attributes, orient='index')

In [8]:
# Fonction pour aplatir un sous-dictionnaire
def flatten_subdict(subdict, prefix=''):
    flat_dict = {}
    for key, value in subdict.items():
        if isinstance(value, dict):
            flat_dict.update(flatten_subdict(value, prefix + key + '_'))
        else:
            flat_dict[prefix + key] = value
    return flat_dict

In [9]:
# Créez un dictionnaire aplatit pour chaque sous-dossier
flattened_data = {}
for sous_dossier, subdict in dico_sequences_attributes.items():
    flattened_data[sous_dossier] = flatten_subdict(subdict)

# Créez un DataFrame à partir du dictionnaire aplatit
df = pd.DataFrame.from_dict(flattened_data, orient='index')

# Réinitialisez l'index pour avoir un 'Sous-dossier' en tant que colonne
df.reset_index(inplace=True)
df.rename(columns={'index': 'Sous-dossier'}, inplace=True)

# Affichez le DataFrame résultant

In [10]:
df = df.rename(columns={'Sous-dossier': 'Sequences'})

In [11]:
df

,Sequences,Properties_Prio recon,Properties_Load images to viewer,Properties_Inline movie,Properties_Auto store images,Properties_Load images to stamp segments,Properties_Load images to graphic segments,Properties_Auto open inline display,Properties_Auto close inline display,Properties_Start measurement without further preparation,...,Diff - Composing_Distortion Corr.,Sequence - Special_Refocus pulse duration,Sequence - Special_Time-shifted MB RF,Geometry - Tim CT_Slabs,Geometry - Tim CT_Slices per slab,Sequence - Part 1_Bandwidth 3,Sequence - Part 1_Bandwidth 4,Sequence - Part 1_Bandwidth 5,Sequence - Part 1_Bandwidth 6,Sequence - Special_Ice ACC algo
0,localizer,On,On,Off,On,On,On,Off,Off,On,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,gre_b0map_sag_p2,Off,On,Off,On,Off,Off,Off,Off,Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,fid_B1Mapping,Off,On,Off,On,Off,Off,On,Off,Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DA_3D_RAD_8TE_bloc1_hamming,Off,On,Off,On,Off,Off,Off,Off,Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DA_3D_RAD_8TE_bloc2_hamming,Off,On,Off,On,Off,Off,Off,Off,Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,DA_3D_RAD_8TE_bloc3_hamming,Off,On,Off,On,Off,Off,Off,Off,Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AAHead_Scout_32ch-head-coil,Off,On,Off,On,Off,On,Off,Off,On,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,tfl_b1map_sag_3mm,Off,On,Off,On,Off,Off,Off,Off,Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,gre_fieldmap_sag_3mm,Off,On,Off,On,Off,Off,Off,Off,Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,t1_mp2rage_sag_0p6,Off,On,Off,On,Off,Off,Off,Off,Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df.to_csv('/home/hdary/sequence_parameters.csv', index=False)

In [13]:
row = df.loc[df['Sequences'] == 't1_mp2rage_sag_0p6']

Get parameters from pdf/xml protocol file from Siemens mri scanner - version Terra

In [14]:
param_list = ["Sequence - Common_Acquisition duration",
            "Routine_FoV read",
            "Routine_FoV phase",
            "Routine_Slice thickness",
            "AcquisitionNumber",
            "Routine_TE",
            "Routine_TR"
            "Routine_Coil elements",
            "Routine_Filter",
            "Contrast - Common_Flip angle",
            "Resolution - Common_Base resolution",
            "Resolution - Common_Phase resolution",
            "Resolution - Common_Phase partial Fourier",
            "Resolution - Filter Image_Distortion Corr.",
            "Routine_Phase enc. dir.",
            "Sequence - Common_Bandwidth"]

In [15]:
dico = {}
for i, r in df.iterrows():
    dico_tmp = {}
    for p in param_list:
        try:
            dico_tmp[p] = r[p]
        except:
            pass
    dico[r['Sequences']] = dico_tmp

In [26]:
dico

{'AAhead_scout': {'ImageType': 'ORIGINAL\\PRIMARY\\M\\NORM\\DIS3D\\DIS2D\\MFSPLIT',
  'Manufacturer': 'Siemens',
  'SeriesDescription': 'AAhead_scout',
  'Manufacturer&rsquo;sModelName': 'MAGNETOM Vida',
  'BodyPartExamined': 'BRAIN',
  'MRAcquisitionType': '3D',
  'SliceThickness': '1.6',
  'RepetitionTime': '3.15',
  'EchoTime': '1.37',
  'NumberofAverages': '1',
  'MagneticFieldStrength': '3',
  'NumberofPhaseEncodingSteps': '160',
  'PixelBandwidth': '539',
  'DeviceSerialNumber': '176082',
  'SoftwareVersion(s)': 'syngo MR XA20',
  'AcquisitionMatrix': '0\\160\\160\\0',
  'In-planePhaseEncodingDirection': 'ROW',
  'FlipAngle': '8',
  'PatientPosition': 'HFS',
  'Rows': '160',
  'Columns': '160',
  'PixelSpacing': '1.625\\1.625'},
 'AAhead_scout_MPR_sag': {'ImageType': 'DERIVED\\PRIMARY\\MPR\\NORM\\DIS3D\\DIS2D\\MFSPLIT',
  'Manufacturer': 'Siemens',
  'SeriesDescription': 'AAhead_scout_MPR_sag',
  'Manufacturer&rsquo;sModelName': 'MAGNETOM Vida',
  'BodyPartExamined': 'BRAIN',
  '

from the first subject in XNAT

In [16]:
import pyxnat
from pyxnat import Interface

In [17]:
xnat_cemerem = Interface(server="https://xnat.crmbm.univ-amu.fr",
                    user='hdary')

In [18]:
project = 'VIDA_BRAIN_EPINOV'

In [19]:
subject = xnat_cemerem.select('/project/{project}/subjects/*'.format(project=project)).get()[0]
experiment_id = xnat_cemerem.select('/project/{project}/subjects/{subject}/experiments/*'.format(project=project, subject=subject)).get()[0]

experiment = xnat_cemerem.select('/project/{project}/subjects/{subject}/experiments/{experiment_id}'.format(project=project, subject=subject, experiment_id=experiment_id))

In [20]:
scans = experiment.scans().get()
scans_id = experiment.scans()
scans_serie_desc = [s.attrs.get('series_description') for s in scans_id]
scans_type = [s.attrs.get('type') for s in scans_id]

In [37]:
uri = '/REST/services/dicomdump'

dico = {}
for i,scan_id in enumerate(scans):
        options = {
        'format': 'csv',
        'src': '/archive/projects/%s/experiments/%s/scans/%s' %
                (project, experiment_id, scan_id),
        'field': ['00189087', '00200105', '00081090', '00080008',
                                '0008103E', '00080070', '00181020', '00181000',
                                '00180015', '00180023', '00180050', '00180080',
                                '00180081', '00180083', '00180087', '00180088',
                                '00180089', '00180095', '00181310', '00181312',
                                '00181314', '00185100', '00189073', '00280010',
                                '00280011', '00280030', '00180024', '00180082']
        }

        from pyxnat.core.jsonutil import csv_to_json
        dicomdump = csv_to_json(
                        str(xnat_cemerem.get(uri, params=options).text))
        dico_tmp = {}
        for d in dicomdump:
                dico_tmp[d['desc'].replace(' ','')] = d['value']

        dico[scans_serie_desc[i]] = dico_tmp

In [38]:
dico

{'AAhead_scout': {'ImageType': 'ORIGINAL\\PRIMARY\\M\\NORM\\DIS3D\\DIS2D\\MFSPLIT',
  'Manufacturer': 'Siemens',
  'SeriesDescription': 'AAhead_scout',
  'Manufacturer&rsquo;sModelName': 'MAGNETOM Vida',
  'BodyPartExamined': 'BRAIN',
  'MRAcquisitionType': '3D',
  'SequenceName': '*fl3d1_ns',
  'SliceThickness': '1.6',
  'RepetitionTime': '3.15',
  'EchoTime': '1.37',
  'NumberofAverages': '1',
  'MagneticFieldStrength': '3',
  'NumberofPhaseEncodingSteps': '160',
  'PixelBandwidth': '539',
  'DeviceSerialNumber': '176082',
  'SoftwareVersion(s)': 'syngo MR XA20',
  'AcquisitionMatrix': '0\\160\\160\\0',
  'In-planePhaseEncodingDirection': 'ROW',
  'FlipAngle': '8',
  'PatientPosition': 'HFS',
  'Rows': '160',
  'Columns': '160',
  'PixelSpacing': '1.625\\1.625'},
 'AAhead_scout_MPR_sag': {'ImageType': 'DERIVED\\PRIMARY\\MPR\\NORM\\DIS3D\\DIS2D\\MFSPLIT',
  'Manufacturer': 'Siemens',
  'SeriesDescription': 'AAhead_scout_MPR_sag',
  'Manufacturer&rsquo;sModelName': 'MAGNETOM Vida',
  '

In [50]:
my_list = ['AcquisitionMatrix',
            'AcquisitionNumber',
            'EchoTime',
            'FlipAngle',
            'ImageType',
            'In-planePhaseEncodingDirection',
            'MRAcquisitionType',
            'MagneticFieldStrength',
            'Manufacturer',
            'ManufacturersModelName',
            'PatientPosition',
            'PercentPhaseFOV',
            'PercentSampling',
            'NumberofPhaseEncodingSteps',
            'PixelBandwidth',
            'CoilString',
            'ReceiveCoilActiveElements',
            'ReconMatrixPE',
            'RepetitionTime',
            'ScanOptions',
            'SequenceName',
            'SliceThickness',
            'SoftwareVersion(s)',
            'PhaseEncodingDirection',
            'SeriesDescription',
            'PixelSpacing',
            'Rows',
            'Columns']

In [51]:
dico_final = {}

In [52]:
for el,val in dico.items():
    dico_final[el] = {}
    for k in my_list:
        try:
            dico_final[el][k] = val[k]
        except:
            pass
    

In [53]:
dico_final

{'AAhead_scout': {'AcquisitionMatrix': '0\\160\\160\\0',
  'EchoTime': '1.37',
  'FlipAngle': '8',
  'ImageType': 'ORIGINAL\\PRIMARY\\M\\NORM\\DIS3D\\DIS2D\\MFSPLIT',
  'In-planePhaseEncodingDirection': 'ROW',
  'MRAcquisitionType': '3D',
  'MagneticFieldStrength': '3',
  'Manufacturer': 'Siemens',
  'PatientPosition': 'HFS',
  'NumberofPhaseEncodingSteps': '160',
  'PixelBandwidth': '539',
  'RepetitionTime': '3.15',
  'SequenceName': '*fl3d1_ns',
  'SliceThickness': '1.6',
  'SoftwareVersion(s)': 'syngo MR XA20',
  'SeriesDescription': 'AAhead_scout',
  'PixelSpacing': '1.625\\1.625',
  'Rows': '160',
  'Columns': '160'},
 'AAhead_scout_MPR_sag': {'EchoTime': '1.37',
  'FlipAngle': '8',
  'ImageType': 'DERIVED\\PRIMARY\\MPR\\NORM\\DIS3D\\DIS2D\\MFSPLIT',
  'MRAcquisitionType': '3D',
  'MagneticFieldStrength': '3',
  'Manufacturer': 'Siemens',
  'PatientPosition': 'HFS',
  'NumberofPhaseEncodingSteps': '160',
  'PixelBandwidth': '539',
  'RepetitionTime': '3.15',
  'SequenceName': '*f

In [54]:
import json

# Specify the file path
file_path = '/home/hdary/protocol_validation.json'

# Save the dictionary as a JSON file
with open(file_path, 'w') as f:
    json.dump(dico_final, f)
